In [1]:
%matplotlib inline

In [2]:
import math

import dimod
import numpy as np
import openjij as oj
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo

# 制約条件
$$ W_j <= \sum_i x_{ij} y_i $$

In [285]:
W = np.array([2.1, 2.5, 1, 3.25], dtype=float)
y = np.array([0.25, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# タスクの数
M = len(W)
# 人の数
N = len(y)

# 目的変数
$$ x \in \{0,1\}^{N \times M} $$

In [286]:
x = Array.create("x", shape=(N, M), vartype="BINARY")
c_ineqs = np.array(
    [LogEncInteger("y" + str(i), (0, int(np.ceil(4 * W[i])))) for i in range(M)]
)

In [292]:
A = np.array([1, 1, 10, 10])
B = 50
# Hy = Constraint(sum(A * (W - y @ x) ** 2), label="Hy")
Hy = Constraint(
    sum(A * (np.ceil(4 * W) + c_ineqs - np.ceil(4 * y) @ x) ** 2), label="Hy"
)
Hx = Constraint(sum(B * (1 - np.sum(x, axis=1)) ** 2), label="Hx")

In [293]:
H = Hx + Hy
model = H.compile()
q, offset = model.to_qubo()

In [294]:
sampler = oj.SQASampler(num_reads=200)
sampleset = sampler.sample_qubo(q)
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [295]:
res_x = np.zeros(x.shape)
for i in range(N):
    for j in range(M):
        res_x[i, j] = decoded_sample.array("x", (i, j))

In [296]:
res_ineq = np.zeros(M)
for j in range(M):
    res_ineq[j] = sum(
        2**k * v
        for k, v in [
            (elem, decoded_sample.array("y" + str(j), elem))
            for elem in range(math.ceil(math.log2(W[j])))
        ]
    )

In [297]:
res_ineq

array([1., 1., 0., 3.])

In [298]:
res_x.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [301]:
np.ceil(4 * W)

array([ 9., 10.,  4., 17.])

In [299]:
np.ceil(4 * y) @ res_x

array([10., 12.,  5., 20.])

In [300]:
np.ceil(4 * W) - np.ceil(4 * y) @ res_x

array([-1., -2., -1., -3.])

In [282]:
res_x

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [4]:
W = 20
c = {0: 5, 1: 7, 2: 2, 3: 1, 4: 4, 5: 3}
w = {0: 8, 1: 10, 2: 6, 3: 4, 4: 5, 5: 3}
N = len(w)

In [35]:
E = 3

In [36]:
x = Array.create("x", shape=(N), vartype="BINARY")
y = LogEncInteger("y", (0, W))
z = LogEncInteger("z", (0, E))

In [37]:
key1 = max(c, key=lambda k: c[k])
B = 40
A = 10 * B * c[key1]

HA1 = Constraint(A * (W - sum(w[a] * x[a] for a in range(N)) - y) ** 2, label="HA1")
HA2 = Constraint(A * (E - sum(x) - z) ** 2, label="HA2")

HB = -B * sum(c[a] * x[a] for a in range(N))

In [38]:
H = HA1 + HA2 + HB
Q = H
model = Q.compile()
q, offset = model.to_qubo()

sampler = oj.SQASampler(num_reads=100)
sampleset = sampler.sample_qubo(q)
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [39]:
sum(
    2**k * v
    for k, v in [
        (elem, decoded_sample.array("z", elem))
        for elem in range(math.ceil(math.log2(E)))
    ]
)

1

In [40]:
print()
print("[Results]")
print()
print("decoded_sample.sample:")
print(decoded_sample.sample)
print()
print("x (選ばれた宝物) :")

treasures = ["A", "B", "C", "D", "E", "F", "G"]
weight = 0
cost = 0

for k in range(N):
    if decoded_sample.array("x", k) != 0:
        print("宝物" + treasures[k])
        weight += w[k]
        cost += c[k]


sol_y = sum(
    2**k * v
    for k, v in [
        (elem, decoded_sample.array("y", elem))
        for elem in range(math.ceil(math.log2(W)))
    ]
)

print()
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("合計の重さ : " + str(weight / 10) + "kg")
print("合計の価格 : $" + str(cost) + ",000")


[Results]

decoded_sample.sample:
{'y[1]': 1, 'x[0]': 1, 'y[0]': 0, 'x[1]': 1, 'x[2]': 0, 'x[3]': 0, 'x[4]': 0, 'x[5]': 0, 'y[2]': 0, 'y[3]': 0, 'y[4]': 0, 'z[0]': 1, 'z[1]': 0}

x (選ばれた宝物) :
宝物A
宝物B

スラック変数Y = 2

broken
{}
合計の重さ : 1.8kg
合計の価格 : $12,000


In [18]:
import dimod

print("[Inputs]")
print()
print("W (ナップサックの容量) : " + str(W / 10) + "kg")
print("N (宝物の数): " + str(N))
print()
print("weight list")
print(w)
print()
print("cost list")
print(c)
print()
print("A : " + str(A))
print("B : " + str(B))

H = HA + HB
Q = H
model = Q.compile()
q, offset = model.to_qubo()

sampleset = dimod.ExactSolver().sample_qubo(q)
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")
print()
print("[Results]")
print()
print("decoded_sample.sample:")
print(decoded_sample.sample)
print()
print("x (選ばれた宝物) :")

treasures = ["A", "B", "C", "D", "E", "F", "G"]
weight = 0
cost = 0

for k in range(N):
    if decoded_sample.array("x", k) != 0:
        print("宝物" + treasures[k])
        weight += w[k]
        cost += c[k]


sol_y = sum(
    2**k * v
    for k, v in [
        (elem, decoded_sample.array("y", elem))
        for elem in range(math.ceil(math.log2(W)))
    ]
)

print()
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("合計の重さ : " + str(weight / 10) + "kg")
print("合計の価格 : $" + str(cost) + ",000")

[Inputs]

W (ナップサックの容量) : 2.0kg
N (宝物の数): 6

weight list
{0: 8, 1: 10, 2: 6, 3: 4, 4: 5, 5: 3}

cost list
{0: 5, 1: 7, 2: 2, 3: 1, 4: 4, 5: 3}

A : 2800
B : 40

[Results]

decoded_sample.sample:
{'y[1]': 1, 'x[0]': 0, 'y[0]': 0, 'x[1]': 1, 'x[2]': 0, 'x[3]': 0, 'x[4]': 1, 'x[5]': 1, 'y[2]': 0, 'y[3]': 0, 'y[4]': 0}

x (選ばれた宝物) :
宝物B
宝物E
宝物F

スラック変数Y = 2

broken
{}
合計の重さ : 1.8kg
合計の価格 : $14,000
